In [5]:
import sys
sys.path.append('/jet/home/cpestell/lab3-1')
from ridge_utils.DataSequence import DataSequence


In [6]:
import sys
sys.path.append('..') 
from ridge_utils.DataSequence import DataSequence
import torch
import pickle
import os
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from encoder import Encoder
from data import TextDataset
from train_encoder import train_bert

/jet/home/cpestell/.conda/envs/env_214/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train Encoder

In [11]:
# Load the podcast text data
data_path = "/ocean/projects/mth240012p/shared/data"
raw_text_path = os.path.join(data_path, "raw_text.pkl")

with open(raw_text_path, "rb") as f:
    raw_text = pickle.load(f)

print(f"Loaded {len(raw_text)} podcast stories")

# Flatten all the words from all stories into a single list
all_sentences = []
for story in raw_text.values():
    all_sentences.extend(story.data)

# Tokenizer & Dataset
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
dataset = TextDataset(all_sentences, tokenizer, max_len=32)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create encoder model
device = "cuda" if torch.cuda.is_available() else "cpu"
encoder = Encoder(vocab_size=tokenizer.vocab_size)
encoder = encoder.to(device)

# Train the model using masked language modeling
train_bert(
    model=encoder,
    dataloader=dataloader,
    tokenizer=tokenizer,
    epochs=20, # we can change this 
    lr=5e-4, # same as well 
    device=device
)

# Save pretrained encoder weights
torch.save(encoder.state_dict(), "pretrained_encoder.pt")
print("Pretrained encoder saved as pretrained_encoder.pt")

/var/tmp/ipykernel_61984/3662475046.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  raw_text = pickle.load(f)


Loaded 109 podcast stories


OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

# Generate All Embeddings 

In [ ]:
import torch
import numpy as np
import pickle, os
from transformers import BertTokenizerFast
from encoder import Encoder
from preprocessing import downsample_word_vectors, make_delayed

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = BertTokenizerFast.from_pretrained("/jet/home/cpestell/bert-base-uncased")  # your local copy
encoder = Encoder(vocab_size=tokenizer.vocab_size)
encoder.load_state_dict(torch.load("pretrained_encoder.pt", map_location=device))
encoder.to(device)
encoder.eval()

# Load raw text
data_path = "/ocean/projects/mth240012p/shared/data"
with open(os.path.join(data_path, "raw_text.pkl"), "rb") as f:
    raw_text = pickle.load(f)

story_names = list(raw_text.keys())

# Step 1: Generate encoder embeddings
encoder_vectors = {}
for story in story_names:
    words = raw_text[story].data
    inputs = tokenizer(words, padding=True, truncation=True, return_tensors="pt", max_length=64)

    input_ids = inputs["input_ids"].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        hidden = encoder(input_ids, token_type_ids, attention_mask)

    # Mean over tokens (seq_len)
    sentence_embeddings = hidden.mean(dim=1).cpu().numpy()
    encoder_vectors[story] = sentence_embeddings
    print(f"{story}: Encoder shape = {encoder_vectors[story].shape}")

# Step 2: Downsample
wordseqs = raw_text
downsampled_encoder = downsample_word_vectors(story_names, encoder_vectors, wordseqs)

# Step 3: Trim
X_encoder_trimmed = {}
for story in downsampled_encoder:
    X_encoder_trimmed[story] = downsampled_encoder[story][5:-10, :]
    print(f"Trimmed shape for {story}:", X_encoder_trimmed[story].shape)

# Step 4: Create lagged features
X_encoder_lagged = {}
for story in X_encoder_trimmed:
    X_encoder_lagged[story] = make_delayed(X_encoder_trimmed[story], delays=range(1, 5))
    print(f"Lagged shape for {story}:", X_encoder_lagged[story].shape)


# Modelling and Evaluation

In [ ]:
import numpy as np
import os
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

# --------- Setup ---------
subject2_path = os.path.join(data_path, "subject2")
X_train = []
Y_train = []

for story in train_stories:
    if story in X_encoder_lagged:
        bold_path = os.path.join(subject2_path, f"{story}.npy")
        if os.path.exists(bold_path):
            Y = np.load(bold_path)
            X = X_encoder_lagged[story]
            min_len = min(X.shape[0], Y.shape[0])
            X_train.append(X[:min_len])
            Y_train.append(Y[:min_len])
        else:
            print(f"Missing Y file for: {story}")

X_train = np.vstack(X_train)
Y_train = np.vstack(Y_train)
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

# --------- Clean NaNs ---------
mask = ~np.isnan(Y_train).any(axis=1)
X_train_clean = X_train[mask]
Y_train_clean = Y_train[mask]

# --------- Cross Validation ---------
ridge = Ridge(alpha=100.0)
cv_scores = cross_val_score(ridge, X_train_clean, Y_train_clean, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean cross-validation score: {np.mean(cv_scores):.4f}")

# --------- Train Model ---------
ridge.fit(X_train_clean, Y_train_clean)

# --------- Evaluate on Test Set ---------
X_test = []
Y_test = []

for story in test_stories:
    if story in X_encoder_lagged:
        bold_path = os.path.join(subject2_path, f"{story}.npy")
        if os.path.exists(bold_path):
            Y = np.load(bold_path)
            X = X_encoder_lagged[story]
            min_len = min(X.shape[0], Y.shape[0])
            X_test.append(X[:min_len])
            Y_test.append(Y[:min_len])
        else:
            print(f"Missing Y file for: {story}")

X_test = np.vstack(X_test)
Y_test = np.vstack(Y_test)

# Clean NaNs from test
mask = ~np.isnan(Y_test).any(axis=1)
X_test_clean = X_test[mask]
Y_test_clean = Y_test[mask]

# Predict
Y_pred = ridge.predict(X_test_clean)

# --------- Compute Correlation Coefficients (CCs) ---------
def compute_voxel_ccs(y_true, y_pred):
    ccs = []
    for i in range(y_true.shape[1]):
        try:
            cc, _ = pearsonr(y_true[:, i], y_pred[:, i])
        except:
            cc = np.nan
        ccs.append(cc)
    return np.array(ccs)

ccs_encoder = compute_voxel_ccs(Y_test_clean, Y_pred)

print(f"Mean CC: {np.mean(ccs_encoder):.4f}")
print(f"Median CC: {np.median(ccs_encoder):.4f}")
print(f"Top 1% CC: {np.percentile(ccs_encoder, 99):.4f}")
print(f"Top 5% CC: {np.percentile(ccs_encoder, 95):.4f}")

# --------- Visualization ---------
sns.set(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(12, 6))

sns.histplot(ccs_encoder[~np.isnan(ccs_encoder)], bins=100, kde=True, color="#1f77b4", edgecolor="black", stat="count")
plt.axvline(np.mean(ccs_encoder), color='red', linestyle='--', label=f"Mean: {np.mean(ccs_encoder):.3f}")
plt.axvline(np.median(ccs_encoder), color='green', linestyle='-.', label=f"Median: {np.median(ccs_encoder):.3f}")
plt.title("Encoder Correlation Coefficient (CC) Distribution (5000 Voxels)", fontsize=16, weight="bold")
plt.xlabel("Correlation Coefficient", fontsize=13)
plt.ylabel("Number of Voxels", fontsize=13)
plt.legend()
plt.tight_layout()

# Save plot
save_dir = os.path.join("..", "results")
os.makedirs(save_dir, exist_ok=True)
plot_path = os.path.join(save_dir, "cc_distribution_encoder_subject2_5000vox.png")
plt.savefig(plot_path, dpi=300)
plt.show()

print(f"Plot saved to: {plot_path}")
